In [1]:
import warnings
warnings.filterwarnings('ignore')

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import PPBuilder
from Bio.PDB.vectors import calc_dihedral

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, math

import urllib.request, urllib.error
import nltk

def degrees(rad):
    return (rad * 180) / math.pi

def phi_psi_omega_to_abego(phi, psi, omega):
    #if np.isnan(psi): return ‘O’
    #if np.isnan(omega): omega = 180
    #if np.isnan(phi): phi=90
    
    if np.isnan(phi) or np.isnan(psi) or np.isnan(omega): 
        return 'X'
    
    if abs(omega) < 90:
        return 'O'
    
    elif phi > 0:
        if -100.0 <= psi < 100:
            return 'G'
        else:
            return 'E'
        
    else:
        if -75.0 <= psi < 50:
            return 'A'
        else:
            return 'B'
        
    return 'X'

In [319]:
df = pd.read_csv('2018-06-06-ss.cleaned.csv') # read in CSV

# first 200 pdb ids
input_pdbs = df['pdb_id'][:100].values.T

# print(input_pdbs)

In [206]:
files = []

for pdb in input_pdbs:
    
    try:
        if pdb not in os.listdir((os.getcwd() + '/pdb_files/')):
            filename = urllib.request.urlretrieve('https://files.rcsb.org/download/{}.pdb'.format(pdb), pdb + '.pdb')
            path = os.path.join(os.getcwd(), pdb + '.pdb')
            os.rename(path, os.getcwd() + '/pdb_files/' + pdb + '.pdb') 
            files.append(filename[0])
        else:
            files.append(pdb)
        
    except urllib.error.HTTPError:
        continue    
        
files = set(files)

In [207]:
# output set
abegopatterns = []

# input set
seqs = []

cwd = os.getcwd()

for file in files:
        
    phi_psi = []
    nres = []
    
    ran = False
    repeat = True
        
    structure = PDBParser(QUIET = True).get_structure(cwd + "/pdb_files/" + file, cwd + "/pdb_files/" + file)

    for chain in structure:

        polypeptides = PPBuilder().build_peptides(chain)

        for polypeptide in polypeptides:

            ran = True
            
            # a list of polypeptide chain lengths
            nres.append(len(polypeptide))
            
            if len(nres) > 1:
                nres[-1] = nres[-1] + nres[-2]
            
            phi_psi += polypeptide.get_phi_psi_list()  
            
            if polypeptide.get_sequence() not in seqs:
                repeat = False # don't want duplicate sequences
                seqs.append(polypeptide.get_sequence())

            break # only the first subunit for now
            
        break
    
    if not(ran) or repeat:
        continue
        
    phi_psi_omega = []

    residues = [res for res in structure.get_residues()]

    for i in range(len(residues) - 1):

        if (i + 1) in nres:
            omega = None
            break

        else:
            try:
                a1 = residues[i]['CA'].get_vector()
                a2 = residues[i]['C'].get_vector()
                a3 = residues[i + 1]['N'].get_vector()
                a4 = residues[i + 1]['CA'].get_vector()
                
                omega = calc_dihedral(a1,a2,a3,a4)
                
                phi_psi_omega.append((phi_psi[i][0], phi_psi[i][1], omega))
                
            except KeyError:
                # phi_psi_omega.append((phi_psi[i][0], phi_psi[i][1], None))
                # seqs.pop()
                continue
    
    # last triplet tuple
    phi_psi_omega.append((phi_psi[-1][0], phi_psi[-1][1], None))
    
    # ABEGO str
    abego = ""

    for phi, psi, omega in phi_psi_omega: 
        if phi != None and psi != None and omega != None:
            abego += phi_psi_omega_to_abego(degrees(phi), degrees(psi), degrees(omega))
        
    abegopatterns.append(abego)

In [208]:
print(len(seqs), len(abegopatterns))

for i in range(len(seqs)):
    seqs[i] = str(seqs[i])

def seq2ngrams(seqs, n=3):
    return np.array([[seq[i:i+n] for i in range(len(seq))] for seq in seqs])

input_grams = seq2ngrams(seqs)

3564 3564


In [209]:
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import hashing_trick

from keras.utils import to_categorical

# encoder, turns sequence into a fixed vector of numbers
# hash function is a possibility
tokenizer_encoder = Tokenizer() # Tokenizer Class Instance

tokenizer_encoder.fit_on_texts(input_grams) # tokenize the input_grams, updates internal unique vocabulary

input_data = tokenizer_encoder.texts_to_sequences(input_grams) # assigns the text a number

input_data = pad_sequences(input_data, maxlen=750, padding='post')

# decoder
tokenizer_decoder = Tokenizer(char_level = True) # every character will be treated as a token because it's ABEGO

tokenizer_decoder.fit_on_texts(abegopatterns) 

target_data = tokenizer_decoder.texts_to_sequences(abegopatterns)

target_data = pad_sequences(target_data, maxlen=750, padding='post')

target_data = to_categorical(target_data) # oneHotEncoder

input_data.shape, target_data.shape

((3564, 750), (3564, 750, 6))

In [ ]:
letters = np.array([[letter for letter in abego if (letter != None)] for abego in abegopatterns])
df = pd.DataFrame(columns=range(len(letters)))
for i in range(len(letters)):
    df[i] = pd.Series(letters[i])
    
cat_encoder = OneHotEncoder()
cat_encoder.fit(df[[0]])

cat_encoder.categories_
practice = []
for i in range(len(letters)):
    practice.append(cat_encoder.transform(df[[i]]).toarray())

In [ ]:
# one-hot encoding
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
abego_cat_1hot = cat_encoder.fit_transform(df)
cat_encoder.categories_

In [304]:
from keras.models import Input, Model
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional

maxlen_seq = 750

n_words = len(tokenizer_encoder.word_index) + 1 # Number of Possible Amino Acids

n_tags = len(tokenizer_decoder.word_index) + 1 # Possible ABEGO Patterns

input_seq = Input(shape = (maxlen_seq, )) 

x = tf.keras.layers.Embedding(input_dim = n_words, output_dim = maxlen_seq, input_length = maxlen_seq)(input_seq)

x = tf.keras.layers.Bidirectional(LSTM(units = 64, return_sequences = True, recurrent_dropout=0.1))(x)

# dense means fully connected
output = tf.keras.layers.TimeDistributed(Dense(n_tags, activation = "softmax"))(x) 

model = Model(input_seq, output)

model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 750)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 750, 750)          6281250   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 750, 128)          417280    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 750, 6)            774       
Total params: 6,699,304
Trainable params: 6,699,304
Non-trainable params: 0
_________________________________________________________________


In [86]:
maxlen_seq = 750

n_words = len(tokenizer_encoder.word_index) + 1 # Number of Possible Amino Acids

n_tags = len(tokenizer_decoder.word_index) + 1 # Possible ABEGO Patterns

new_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(maxlen_seq,)),
    keras.layers.Embedding(input_dim = n_words, output_dim = maxlen_seq, input_length = maxlen_seq),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(n_tags, activation="softmax"))
#   keras.layers.Dense(12000, activation="relu"),
#   keras.layers.Embedding(n_words, 750, input_length = 32)
#   keras.layers.Dense(1, input_shape=(5,), activation="softmax")
])

new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 750, 750)          2749500   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 750, 200)          680800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 750, 6)            1206      
Total params: 3,431,506
Trainable params: 3,431,506
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.model_selection import train_test_split
from keras.metrics import categorical_accuracy

# model.compile defines the loss function, optimizer, and metrics
# first metric is Keras provided, second metric is custom metric
model.compile(optimizer="RMSprop", loss="categorical_crossentropy", metrics = ["accuracy"]) 

X_train, X_test, y_train, y_test = train_test_split(input_data, target_data, test_size = .4, random_state=0)

model.fit(X_train, y_train, epochs=5)

In [246]:
reverse_decoder_index = {value:key for key, value in tokenizer_decoder.word_index.items()}
reverse_encoder_index = {value:key for key, value in tokenizer_encoder.word_index.items()}

In [ ]:
test_predictions = model.predict(X_test)

In [300]:
seq = ''
pred = ''
for amino in X_test[0]:
    if amino != 0:
        print(amino)
        seq += reverse_encoder_index[amino]
    else:
        break
        
for letter in test_predictions[0]:
    if np.argmax(letter) != 0:
        pred += reverse_decoder_index[np.argmax(letter)]
    else:
        break

test = ''
for letter in y_test[0]:
    if np.argmax(letter) != 0:
        test += reverse_decoder_index[np.argmax(letter)]
    else:
        break

print("Sequence: \n", seq.upper())
print("Actual: \n", test.upper())
print("Predicted: \n", pred.upper())

1806
8112
1519
Sequence: 
 NIIIII
Actual: 
 A
Predicted: 
 B


In [66]:
train_predictions = new_model.predict(X_train)

In [83]:
train_pred = ''
for letter in train_predictions[13]:
    if np.argmax(letter) != 0:
        train_pred += reverse_decoder_index[np.argmax(letter)]
train_pred

'bbbbbbbbbbbaaaaaaaaaaaa'

In [280]:
# for prediction in predictions:
#     for letter in prediction:
#         if np.argmax(letter) == 5:
#             pred += reverse_decoder_index[np.argmax(letter)]
len(predictions)

1426